# Sanity Checks

In [1]:
import os 
import sys
import glob

sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')

import itertools
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from tqdm.notebook import tqdm
from collections import defaultdict, Counter
from sklearn.metrics import accuracy_score, f1_score

from src.data.make_dataset import build_dataset
from src.models.word2vec.User2Subreddit import User2Subreddit

from sklearn.metrics import auc, roc_curve

### Load In Comment Affiliations

In [2]:
year_month = '2019-09'
comment_directory = '/shared/0/projects/reddit-political-affiliation/data/comment-affiliations/*' + year_month + ".tsv"
files = glob.glob(comment_directory)

user_to_politics = {}

for fname in files:
     with open(fname, 'r') as f:
        for line in f:
            user, politics = line.split('\t')
            user_to_politics[user] = politics.strip().lower()

In [3]:
dems, reps = 0, 0
for user, politics in user_to_politics.items():
    if politics == "democrat":
        dems += 1
    else:
        reps += 1

print(dems, reps)

5850 2626


## Distribution of Validation Set

In [4]:
# Load the dataset in
year_month = '2019-09'

network_path = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/' + year_month + '_filtered.tsv'
flair_directory = '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/' + year_month + '.tsv'
comment_directory = '/shared/0/projects/reddit-political-affiliation/data/comment-affiliations/*' + year_month + ".tsv"
print(comment_directory)

dataset, training, validation, pol_validation, vocab = build_dataset(network_path, flair_directory, comment_directory)

/shared/0/projects/reddit-political-affiliation/data/comment-affiliations/*2019-09.tsv



Building vocab from file:  20%|█▉        | 5443839/27368767 [00:14<01:08, 317942.19it/s]


Building vocab from file:  40%|████      | 11015037/27368767 [00:28<00:26, 616432.65it/s]


Building vocab from file:  60%|█████▉    | 16405381/27368767 [00:41<00:17, 644084.79it/s]


Building vocab from file:  78%|███████▊  | 21465651/27368767 [00:56<00:10, 561195.87it/s]


Building vocab from file:  95%|█████████▍| 25960877/27368767 [01:11<00:03, 440107.76it/s]


Converting data to PyTorch:   0%|          | 385/5714563 [00:00<24:49, 3835.92it/s]

Length of vocab: 5777375
User count: 5714563
Subreddit count: 62812
User to politic counts: 3322
[('JobieWanKenobi', Counter({'republican': 1})), ('Iowa_Hawkeye', Counter({'republican': 2})), ('Blue387', Counter({'republican': 2})), ('_Hospitaller_', Counter({'republican': 1})), ('RogueHippie', Counter({'republican': 1})), ('SuicidalTendies', Counter({'republican': 1})), ('Znut55', Counter({'republican': 1})), ('rollingrock16', Counter({'republican': 1})), ('Immigrants_go_home', Counter({'republican': 1})), ('nukesiliconvalleyplz', Counter({'republican': 1}))]
Saw political affiliations for 3321 users from flairs
Number of democrats: 408
Number of republicans: 2913
Saw political affiliations for 8476 users from comments
Number of democrats: 5850
Number of republicans: 2626
User to politics training size: {}: 10487
User to politics validation size: {}: 1165


Converting data to PyTorch: 100%|██████████| 5714563/5714563 [08:21<00:00, 11395.92it/s]


Train size: 147791342 Validation size: 16421260


In [5]:
reps, dems = 0, 0

for user, label in pol_validation.items():
    if label == 0:
        dems += 1
    else:
        reps += 1

total = reps + dems
print("Percent republicans: {}".format(reps/total))
print("Percent democrats: {}".format(dems/total))

Percent republicans: 0.46008583690987126
Percent democrats: 0.5399141630901287


## Sklearn Dummy Classifiers

In [6]:
user_ids, pol_labels = [], []

for user, pol_label in pol_validation.items():
    try:
        user_ids.append(dataset.user_to_idx[user])
        pol_labels.append(pol_label)
    except KeyError:
        pass

In [7]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(user_ids, pol_labels)
y_pred = dummy_clf.predict(user_ids)
print("Dummy mean accuracy: {}".format(dummy_clf.score(user_ids, pol_labels)))
print("F1 Score (Macro): {}".format(f1_score(y_pred, pol_labels, average='macro')))

Dummy mean accuracy: 0.5352380952380953
F1 Score (Macro): 0.34863523573200994


/opt/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Single Month Analysis Not Using Predictions

In [8]:
def read_political_affiliations(files):
    user_to_politic_counts = defaultdict(Counter)

    for fname in tqdm(files):
        with open(fname, 'rt') as f:
            for line in f:
                user, politics, freq = line.split('\t')
                user_to_politic_counts[user][politics] += int(freq)

    print("User to politic counts: " + str(len(user_to_politic_counts)))
    print(list(user_to_politic_counts.items())[:10])

    user_to_politics = {}
    for u, pc in user_to_politic_counts.items():
        if len(pc) > 1:
            continue
        user_to_politics[u] = list(pc.keys())[0]

    print('Saw political affiliations for %d users' % len(user_to_politics))
    return convert_affiliations_to_binary(user_to_politics)


def convert_affiliations_to_binary(user_to_politics):
    for user, politics in user_to_politics.items():
        if politics == "Democrat":
            user_to_politics[user] = 0
        else:
            user_to_politics[user] = 1

    return user_to_politics

flair_directory = "/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/" + year_month + ".tsv"
flair_files = glob.glob(flair_directory)
user_to_politics = read_political_affiliations(flair_files)


User to politic counts: 3322
[('JobieWanKenobi', Counter({'Republican': 1})), ('Iowa_Hawkeye', Counter({'Republican': 2})), ('Blue387', Counter({'Republican': 2})), ('_Hospitaller_', Counter({'Republican': 1})), ('RogueHippie', Counter({'Republican': 1})), ('SuicidalTendies', Counter({'Republican': 1})), ('Znut55', Counter({'Republican': 1})), ('rollingrock16', Counter({'Republican': 1})), ('Immigrants_go_home', Counter({'Republican': 1})), ('nukesiliconvalleyplz', Counter({'Republican': 1}))]
Saw political affiliations for 3321 users


In [12]:
# Distribution
reps, dems = 0, 0
for k, v in user_to_politics.items():
    if v == 1:
        reps += 1
    else:
        dems += 1
        
total = reps + dems
print("Percent republicans: {}".format(reps/total))
print("Percent democrats: {}".format(dems/total))

Percent republicans: 0.8771454381210478
Percent democrats: 0.12285456187895212


## Polarized Subreddits

In [10]:
user_subreddits = dataset.user_subreddits
subreddit_scores = defaultdict(lambda:0)
subreddit_counts = Counter()
subreddit_users = defaultdict(set)

MIN_POST_THRESHOLD = 20

for user, score in user_to_politics.items():
    subreddits = user_subreddits[user]
    for sub in subreddits:
        if sub[2:4] != 'u_':
            subreddit_scores[sub] += score
            subreddit_counts[sub] += 1
            subreddit_users[sub].add(user)

norm_sub_scores = {}

for sub, score in subreddit_scores.items():
    count = subreddit_counts[sub]
    if count >= 20:
        norm_sub_scores[sub] = score / count
    
sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=True)}
top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

print("Top 25 right leaning subreddits")

for sub, score in top_results.items():
    print(sub, score, subreddit_counts[sub])

Top 25 right leaning subreddits
r/gundeals 1.0 48
r/walkaway 1.0 39
r/NOWTTYG 1.0 30
r/drumpfisfinished 1.0 47
r/Catholicism 1.0 32
r/DrainTheSwamp 1.0 63
r/TrueChristian 1.0 21
r/TheNewRight 1.0 56
r/DeclineIntoCensorship 1.0 21
r/WhereAreAllTheGoodMen 1.0 22
r/keto 1.0 24
r/Republican 1.0 96
r/MGTOW 1.0 50
r/HateCrimeHoaxes 1.0 21
r/metacanada 1.0 27
r/EarthPorn 1.0 25
r/kotakuinaction2 1.0 36
r/whatisthisthing 1.0 21
r/modernwarfare 1.0 43
r/clevercomebacks 1.0 20
r/aviation 1.0 24
r/The_Donald 0.9988276670574443 1706
r/ShitPoliticsSays 0.9925925925925926 135
r/Conservative 0.9885057471264368 696
r/AskThe_Donald 0.9867549668874173 151


In [11]:
sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=False)}
top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

print("Top 25 left leaning subreddits")

for sub, score in top_results.items():
    print(sub, score, subreddit_counts[sub])

Top 25 left leaning subreddits
r/LateStageImperialism 0.043478260869565216 23
r/AskALiberal 0.1896551724137931 232
r/CanadaPolitics 0.2549019607843137 51
r/neoliberal 0.35135135135135137 37
r/ukpolitics 0.4339622641509434 53
r/Gamingcirclejerk 0.52 25
r/ChapoTrapHouse 0.55 20
r/AskAnAmerican 0.5675675675675675 74
r/AskConservatives 0.6 45
r/Ask_Politics 0.6 20
r/unitedkingdom 0.6086956521739131 23
r/menwritingwomen 0.6190476190476191 21
r/SubredditDrama 0.625 32
r/liberalgunowners 0.625 24
r/SelfAwarewolves 0.6428571428571429 28
r/ToiletPaperUSA 0.6428571428571429 28
r/SandersForPresident 0.6585365853658537 41
r/changemyview 0.6619718309859155 71
r/PoliticalDiscussion 0.6666666666666666 54
r/LateStageCapitalism 0.6666666666666666 21
r/TheRightCantMeme 0.696969696969697 33
r/tumblr 0.7 20
r/ThatsInsane 0.7 20
r/moderatepolitics 0.7 30
r/ENLIGHTENEDCENTRISM 0.7073170731707317 41


## Major Subreddit Distributions

In [ ]:
import seaborn as sns

major_subreddits = ['r/politics', 'r/pics', 'r/AskReddit', 'r/Conservative', 'r/Liberal']

def plot_sub_scores(subreddit, subreddit_users):
    sns.set_theme(style="darkgrid")
    scores = []
    
    for user in subreddit_users[subreddit]:
        score = user_to_politics[user]
        scores.append(score)
        
    sns.displot(scores).set(title=subreddit)
#     sns.plt.xlim(0, 1)
#     sns.plt.set_title(subreddit)
#     sns.plt.show()

for sub in major_subreddits:
    plot_sub_scores(sub, subreddit_users)

# SVD On Subreddit Embeddings

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Load in the model
PATH = '/shared/0/projects/reddit-political-affiliation/working-dir/word2vec-outputs/' + year_month + '/9.pt'
embedding_dim = 50
model = User2Subreddit(dataset.num_users(), embedding_dim, len(dataset.subreddit_to_idx))
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
print(model.eval())

# Compute SVD on the subreddit embeddings
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
X = model.v_embeddings.weight.detach().numpy()
svd.fit(X)

In [ ]:
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

# Multi-month Analysis Not Using Predictions

In [ ]:
# Load in multiple months
year_months = ['2019-04', '2019-05', '2019-06']
models = {}
datasets = {}
for year_month in year_months:
    network_path = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/' + year_month + '_filtered.tsv'
    flair_directory = '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/' + year_month + '.tsv'

    dataset, training, validation, pol_validation, vocab = build_dataset(network_path, flair_directory)
    PATH = '/shared/0/projects/reddit-political-affiliation/working-dir/word2vec-outputs/' + year_month + '/9.pt'
    embedding_dim = 50
    model = User2Subreddit(dataset.num_users(), embedding_dim, len(dataset.subreddit_to_idx))
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
    models[year_month] = model
    datasets[year_month] = dataset

In [ ]:
for year_month in year_months:
    dataset = datasets[year_month]
    model = models[year_month]
    user_subreddits = dataset.user_subreddits
    subreddit_scores = defaultdict(lambda:0)
    subreddit_counts = Counter()
    subreddit_users = defaultdict(set)

    MIN_POST_THRESHOLD = 20

    for user, score in user_to_politics.items():
        subreddits = user_subreddits[user]
        for sub in subreddits:
            if sub[2:4] != 'u_':
                subreddit_scores[sub] += score
                subreddit_counts[sub] += 1
                subreddit_users[sub].add(user)

    norm_sub_scores = {}

    for sub, score in subreddit_scores.items():
        count = subreddit_counts[sub]
        if count >= 20:
            norm_sub_scores[sub] = score / count

    sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=True)}
    top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

    print("Top 25 right leaning subreddits for year: {}".format(year_month))

    for sub, score in top_results.items():
        print(sub, score, subreddit_counts[sub])

    sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=False)}
    top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

    print("Top 25 left leaning subreddits for year: {}".format(year_month))

    for sub, score in top_results.items():
        print(sub, score, subreddit_counts[sub])